In [1]:
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pickle
import numpy as np
from keras.models import load_model
import json
import random
import re
factory = StemmerFactory()
stemmer = factory.create_stemmer()
model = load_model('model/models.keras')
stop_words = set(stopwords.words("indonesian"))
intents = json.loads(open('model/intents.json').read())
words = pickle.load(open('model/texts.pkl','rb'))
classes = pickle.load(open('model/labels.pkl','rb'))

In [30]:
with open('model/baku.json') as f:
    normalization_data = json.load(f)
# Normalization
def normalize_word(word):
    return normalization_data.get(word, word)
# NLP
def nlp_steps(sentence):
    sentence = sentence.lower() # Case Folding
    sentence_words = nltk.word_tokenize(sentence) # Tokenization
    sentence_words = [normalize_word(word.lower()) for word in sentence_words] # Normalization
    sentence_words = [re.sub(r'[?!]', '', word) for word in sentence_words] # Menghapus karakter spesial
    sentence_words = [re.sub(r'\d+', '', word) for word in sentence_words] # Menghapus angka
    sentence_words = [word for word in sentence_words if word not in stop_words] # Filtering (menghapus stop words)
    sentence_words = [stemmer.stem(word) for word in sentence_words] # Stemming
    return sentence_words
    # return sentence

# Bag of Words
def bow(sentence, words, show_details=True):
    sentence_words = nlp_steps(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    return(np.array(bag))

# Untuk menghitung probabilitas hasil BoW menggunakan model yang sudah dibuat dan mengurutkan probabilitas tertinggi ke terendah
def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(p.reshape(1, 1, -1))[0]
    ERROR_THRESHOLD = 0.01  # Ambang batas probabilitas
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # results = [[i, r] for i, r in enumerate(res)]
    if not results:
        return [{"intent": "fallback", "probability": "0"}]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

# Untuk memilih jawaban yang sesuai dengan dataset berdasarkan hasil predict tertinggi
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# Untuk menampilkan hasil getResponse    
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [33]:
predict_class("Saya punya gudik dan saya merasa demam", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


[{'intent': 'Demam', 'probability': '0.53078634'},
 {'intent': 'Jadwal', 'probability': '0.04224621'},
 {'intent': 'Malam', 'probability': '0.037585154'},
 {'intent': 'gigitan ular', 'probability': '0.035579387'},
 {'intent': 'Dokter', 'probability': '0.032454737'},
 {'intent': 'Tersedak', 'probability': '0.0304764'},
 {'intent': 'Diare', 'probability': '0.030214284'},
 {'intent': 'Keseleo', 'probability': '0.02906099'},
 {'intent': 'Batuk', 'probability': '0.017715365'},
 {'intent': 'Masalah Pencernaan', 'probability': '0.016060906'},
 {'intent': 'Sore', 'probability': '0.015846314'},
 {'intent': 'Alamat', 'probability': '0.014135708'},
 {'intent': 'kejang', 'probability': '0.01241035'},
 {'intent': 'Siang', 'probability': '0.012111715'},
 {'intent': 'Pendarahan Normal', 'probability': '0.011867524'},
 {'intent': 'gigitan binatang', 'probability': '0.011241016'},
 {'intent': 'Pilek', 'probability': '0.010426451'}]